In [37]:
%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of pyha_analyzer.models.demo_CNN failed: Traceback (most recent call last):
  File "/home/s.perry.543/pyha-analyzer-2.0/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 278, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "/home/s.perry.543/pyha-analyzer-2.0/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 508, in superreload
    update_generic(old_obj, new_obj)
  File "/home/s.perry.543/pyha-analyzer-2.0/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 405, in update_generic
    update(a, b)
  File "/home/s.perry.543/pyha-analyzer-2.0/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/s.perry.543/pyha-analyzer-2.0/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 405, in update_generic
    update(a, b)
  File 

In [38]:
from pyha_analyzer import PyhaTrainer, extractors

# Loading in Data

In [39]:
# Sorry TQ, I don't have these files mounted yet...
# peru132_extr = extractors.Peru132Extractor()
# peru_132_ads = peru132_extr("/data/XC_wav")

In [40]:
# peru_132_ads = peru_132_ads.get_provenance()

In [41]:
birdset_extactor = extractors.Birdset()

hsn_ads = birdset_extactor("HSN")
# per_ads = birdset_extactor("PER")

hsn_ads

DatasetDict({
    train: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 4368
    })
    valid: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 1092
    }

In [42]:
type(hsn_ads)

pyha_analyzer.dataset.AudioDataset

In [43]:
hsn_ads["train"]

Dataset({
    features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
    num_rows: 4368
})

# Online Preprocessing example

Suppose we just wanted spectrograms with no audio preprocessing

In [44]:
hsn_ads["test"][0]["audio"]

{'bytes': None,
 'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e7a5318118cabfab47a509edeb627860a60537535aeea20a19fced4c110d579e/HSN_001_20150708_061805_000_005.ogg'}

In [45]:
hsn_ads

DatasetDict({
    train: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 4368
    })
    valid: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 1092
    }

In [46]:
import librosa
import numpy as np
import torchvision.transforms as transforms


# Perhaps this lives in its own folder? We should define transforms somewhere nice
# ALWAYS TEST ON MANY INPUTS WILL BREAK OTHERWISE
def create_mel_spectrogram_transform(items):
    new_audio = []
    for item_idx in range(len(items["audio"])):
        y, sr = librosa.load(path=items["audio"][item_idx]["path"])
        # padding yippeee
        start = 0
        if y.shape[-1] > (sr * 5):
            start = np.random.randint(0, y.shape[-1] - (sr * 5))
        else:
            y = np.pad(y, (sr * 5) - y.shape[-1])
        pillow_transforms = transforms.ToPILImage()
        new_audio.append(
            np.array(
                pillow_transforms(
                    librosa.feature.melspectrogram(y=y[start : start + (sr * 5)], sr=sr)
                ),
                np.float32,
            )[np.newaxis, ::]
            / 255
        )
    items["audio_in"] = new_audio
    items["audio"] = new_audio
    return items


hsn_ads.set_transform(create_mel_spectrogram_transform)
hsn_ads["train"][[0, 1]]["audio"][0]

array([[[0.11764706, 0.21960784, 0.10588235, ..., 0.6392157 ,
         0.93333334, 0.5294118 ],
        [0.10980392, 0.24313726, 0.56078434, ..., 1.        ,
         0.58431375, 0.92941177],
        [0.03529412, 0.3529412 , 0.4745098 , ..., 0.2627451 ,
         0.42352942, 0.7019608 ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]], shape=(1, 128, 216), dtype=float32)

In [47]:
print(np.array(hsn_ads["train"][2]["audio_in"]).shape)
hsn_ads["train"][[1, 2, 3]]["audio_in"][0].shape

(1, 128, 216)


(1, 128, 216)

# Model Training

As a demo, implementing a model here because we haven't fleshed out the AudioDataset api yet

In the future this exists in `/pyha_analyzer/models/`

In [48]:
from pyha_analyzer.models.demo_CNN import ResnetConfig, ResnetModel

# Going to make notes on anything that should be handled not here
# This is one of these things, this should be handled by potentially pyha_trainer
resnet50d_config = ResnetConfig(
    num_classes=len(hsn_ads["train"].features["ebird_code"].names), input_channels=1
)
model = ResnetModel(resnet50d_config)

In [ ]:
trainer = PyhaTrainer(
    model=model,
    dataset=hsn_ads,
)
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss


#